In [ ]:
import kagglehub
path = kagglehub.dataset_download("ess1004/visa-anomaly-detection")
print("Path to dataset files:", path)

100%|██████████| 1.78G/1.78G [00:51<00:00, 37.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ess1004/visa-anomaly-detection/versions/1


In [ ]:
import kagglehub
path = kagglehub.dataset_download("ipythonx/mvtec-ad")
print("Path to dataset files:", path)

100%|██████████| 4.91G/4.91G [02:05<00:00, 42.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ipythonx/mvtec-ad/versions/2


In [ ]:
import os
from glob import glob
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.notebook import tqdm

print("PyTorch Version:", torch.__version__)

contrastive_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class AggregatedIndustrialDataset(Dataset):
    """
    A custom PyTorch Dataset to aggregate normal images from MVTec-AD and VisA.
    """
    def __init__(self, root_dirs_and_patterns, transform):
        super().__init__()
        self.transform = transform
        self.image_paths = []

        print("🔍 Starting dataset aggregation...")
        for name, root_dir, pattern in root_dirs_and_patterns:
            search_path = os.path.join(root_dir, pattern)
            paths = glob(search_path, recursive=True)
            self.image_paths.extend(paths)
            print(f"  - Found {len(paths)} images in {name}")

        self.image_paths = [p for p in self.image_paths if 'mask' not in p]

        if not self.image_paths:
            raise ValueError("No images found. Check the root directories and patterns.")

        print(f"\n✅ Aggregation complete. Total normal images found: {len(self.image_paths)}\n")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        try:
            image = Image.open(image_path).convert("RGB")
            view1 = self.transform(image)
            view2 = self.transform(image)
            return view1, view2
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return torch.zeros(3, 224, 224), torch.zeros(3, 224, 224)

DATASET_CONFIG = [
    ("MVTec-AD", "/root/.cache/kagglehub/datasets/ipythonx/mvtec-ad/versions/2", "*/train/good/*.png"),
    ("VisA", "/root/.cache/kagglehub/datasets/ess1004/visa-anomaly-detection/versions/1", "*/Data/Images/Normal/*.JPG")
]

try:
    comprehensive_dataset = AggregatedIndustrialDataset(
        root_dirs_and_patterns=DATASET_CONFIG,
        transform=contrastive_transforms
    )

    train_loader = DataLoader(
        comprehensive_dataset,
        batch_size=128,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    print("Verifying the DataLoader by iterating through all batches...")

    for view1_batch, view2_batch in tqdm(train_loader, desc="Loading batches"):
        pass

    print("\nSuccessfully iterated through all batches.")
    print(f"  - Shape of last view 1 batch: {view1_batch.shape}")
    print(f"  - Shape of last view 2 batch: {view2_batch.shape}")


except ValueError as e:
    print(e)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

PyTorch Version: 2.8.0+cu126
🔍 Starting dataset aggregation...
  - Found 3629 images in MVTec-AD
  - Found 9621 images in VisA

✅ Aggregation complete. Total normal images found: 13250

Verifying the DataLoader by iterating through all batches...


Loading batches:   0%|          | 0/104 [00:00<?, ?it/s]


Successfully iterated through all batches.
  - Shape of last view 1 batch: torch.Size([66, 3, 224, 224])
  - Shape of last view 2 batch: torch.Size([66, 3, 224, 224])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

BATCH_SIZE = 32
train_loader = DataLoader(
    comprehensive_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
print(f"DataLoader recreated with a smaller batch size of {BATCH_SIZE}.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class ContrastiveModel(nn.Module):
    def __init__(self, backbone_model):
        super().__init__()
        self.backbone = nn.Sequential(*list(backbone_model.children())[:-1])
        self.projection_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        projection = self.projection_head(features)
        return projection

backbone = models.wide_resnet50_2(weights=models.Wide_ResNet50_2_Weights.IMAGENET1K_V1)
model = ContrastiveModel(backbone).to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
scaler = torch.amp.GradScaler()

num_epochs = 20
temperature = 0.5

print("\n🚀 Restarting contrastive learning fine-tuning...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    
    for view1, view2 in loop:
        view1, view2 = view1.to(device), view2.to(device)
        batch_size = view1.shape[0]
        views = torch.cat([view1, view2], dim=0)

        optimizer.zero_grad()
        
        with torch.amp.autocast(device_type='cuda'):
            projections = model(views)
            projections = nn.functional.normalize(projections, dim=1)
            proj1, proj2 = projections[:batch_size], projections[batch_size:]
            similarity_matrix = torch.matmul(proj1, proj2.T) / temperature
            labels = torch.arange(batch_size).to(device)
            loss = criterion(similarity_matrix, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {avg_loss:.4f}")

print("\n✅ Training complete.")
torch.save(model.backbone.state_dict(), "/content/drive/MyDrive/universal_feature_extractor.pth")
print("💾 Universal feature extractor backbone saved to universal_feature_extractor.pth")

DataLoader recreated with a smaller batch size of 32.
Using device: cuda

🚀 Restarting contrastive learning fine-tuning...


Epoch [1/20]:   0%|          | 0/415 [00:00<?, ?it/s]

Epoch [1/20] - Average Loss: 1.8298


Epoch [2/20]:   0%|          | 0/415 [00:00<?, ?it/s]

Epoch [2/20] - Average Loss: 1.7994


...

Epoch [18/20] - Average Loss: 1.7719


Epoch [19/20]:   0%|          | 0/415 [00:00<?, ?it/s]

Epoch [19/20] - Average Loss: 1.7783


Epoch [20/20]:   0%|          | 0/415 [00:00<?, ?it/s]

Epoch [20/20] - Average Loss: 1.7769

✅ Training complete.
💾 Universal feature extractor backbone saved to universal_feature_extractor.pth
